In [3]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, cm, figure
import pandas as pd
import matplotlib.patheffects as path_effects
from matplotlib.offsetbox import AnchoredText
from matplotlib.animation import FFMpegWriter, FFMpegFileWriter, MovieWriter
from matplotlib import animation, cm, figure
from datetime import datetime
import ffmpeg
import cv2
import pydicom
from scipy import signal
from IPython.display import HTML

import os
import sys


### 

### Import functions from "util.py"

In [4]:
from util import  extract_ultrasound_from_dicom,analyse_flow_from_dicom,get_flow_map, make_animation, apply_time_filter, compose_flow_overlay, apply_alpha,calculate_scores

### Extract pixel values from ultrasound data (3007889_0051.dcm) and time per frame

In [62]:
us_data, timestep = extract_ultrasound_from_dicom("files/3007889_0051.dcm")

### 

# NEW PROGRESS!
######  
# Week 5 - June 7th
* Group frames into windows based on the given window size
* Generate reference windows by taking the mean pixel values of all frames in a window
* Compute sum of squared difference between frames in each window and its reference window
* Write a function that does the above tasks and ouputs the sum of squared differences
#### 
###### Equations:

![SegmentLocal](get_attachment_url.png "equation")


 
### Make a temporary dataset with 10 frames -> `data`
* I created a smaller dataset that consists of 10 frames to avoid computational problems for this task

In [9]:
data = us_data[0:10]

In [10]:
data.shape

(10, 674, 1024)

##### 

### Create `pixel windows` (groups of frames), based on the given window size

In [18]:
# DEFINE WINDOW SIZE AND GROUP FRAMES IN ADVANCE ACCORDING TO WINDOW SIZE:
#### In this case the window size is 2, so window 0 should have <frame 0, frame 1>, window 1 should have <frame 1, frame 2> ....

window_size = 2
pixel_window = []

for i in range(len(data)- window_size):
    window = data[i:i + window_size]
    pixel_window.append(data[i:i + window_size])


In [230]:
#### To test if this worked the way it should, I will test this out using window 5
####       In theory, window 5 should contain <frame 5, frame 6>
####       I created a object called "zipped" that contains <frame 5, frame 6> to test if they match up using 'np.array_equal'

pixel_window[5]

array([[[177, 177, 177, ...,   0,   0,   0],
        [178, 178, 178, ...,   0,   0,   0],
        [174, 190, 169, ...,   0,   0,   0],
        ...,
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0]],

       [[177, 177, 177, ...,   0,   0,   0],
        [178, 178, 178, ...,   0,   0,   0],
        [174, 190, 169, ...,   0,   0,   0],
        ...,
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0]]], dtype=uint8)

In [305]:
zipped = np.stack((data[5], data[6]))

In [306]:
zipped

array([[[177, 177, 177, ...,   0,   0,   0],
        [178, 178, 178, ...,   0,   0,   0],
        [174, 190, 169, ...,   0,   0,   0],
        ...,
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0]],

       [[177, 177, 177, ...,   0,   0,   0],
        [178, 178, 178, ...,   0,   0,   0],
        [174, 190, 169, ...,   0,   0,   0],
        ...,
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0]]], dtype=uint8)

In [233]:
#### The arrays are identical
np.array_equal(zipped, pixel_window[5])

True

# 

### Create `reference window`

In [14]:
# CREATE REFERENCE WINDOWS BY TAKING THE MEAN PIXEL VALUES OF FRAMES IN EACH WINDOW

window_size = 2
reference_window = []

for i in range(len(data)- window_size):
    window = data[i:i + window_size]
    reference_window.append(window.mean(axis = 0))


[[[177 177 177 ...   0   0   0]
  [178 178 178 ...   0   0   0]
  [174 190 169 ...   0   0   0]
  ...
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]]

 [[177 177 177 ...   0   0   0]
  [178 178 178 ...   0   0   0]
  [174 190 169 ...   0   0   0]
  ...
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]]]
[[[177 177 177 ...   0   0   0]
  [178 178 178 ...   0   0   0]
  [174 190 169 ...   0   0   0]
  ...
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]]

 [[177 177 177 ...   0   0   0]
  [178 178 178 ...   0   0   0]
  [174 190 169 ...   0   0   0]
  ...
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]]]
[[[177 177 177 ...   0   0   0]
  [178 178 178 ...   0   0   0]
  [174 190 169 ...   0   0   0]
  ...
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]]



In [323]:
#### Each reference window is the average pixel values of all the frames in that particular window
reference_window[0]

array([[177., 177., 177., ...,   0.,   0.,   0.],
       [178., 178., 178., ...,   0.,   0.,   0.],
       [174., 190., 169., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.]])

#### 

### Compute `sum of squared difference `

In [313]:
# COMPUTE THE SUM OF DIFFERENCE SQUARED BETWEEN EACH WINDOW AND ITS REFERENCE WINDOW

sum_diff_squared = []
#sd = []


for frame, reference in zip(pixel_window, reference_window):
    sum_diff_squared.append(np.sum(np.square(np.subtract(frame, reference))))
    #sd.append(np.std(np.square(np.subtract(frame, reference))))


In [314]:
sum_diff_squared

[4276382.0,
 8173670.5,
 2920876.5,
 2513283.5,
 2124824.5,
 1847891.0,
 1121319.0,
 1132827.0]

In [ ]:
#### A TEST TO SEE WHAT I HAVE DONE ABOVE IS CORRECT
####    If I purposely use reference window 5 as the reference, in theory the sum of squared difference value 
####    should be the same as the above at index 5  (1847891.0)

test = []

for x in pixel_window:
    test.append(np.sum(np.square(np.subtract(x,reference_window[5]))))

In [285]:
test

[164370020.0,
 99401005.0,
 46611862.0,
 22087545.0,
 7045166.0,
 1847891.0,
 4468252.0,
 10071651.0]

In [315]:
#### The values matched up
np.array_equal(test[5], sum_diff_squared[5])

True

### 

### Make this into a `function`

* This function takes 2 arguments: (1) `pixel ultrasound data` (2) `window size` (number of frames in a window)
* The output produces the "sum of squared difference" between frames in each window with its corresponding reference window

* I will also show the time it takes to use this function on different data sizes and window sizes

In [291]:
def steadiness(pixel_data, window_size):
    sum_diff_squared = []
    
    for i in range(len(pixel_data) - window_size):
        pixel_window = pixel_data[i:i + window_size]
        reference_window = pixel_window.mean(axis = 0)
        
        
        sum_diff_squared.append(np.sum(np.square(np.subtract(pixel_window, reference_window))))
        
    return sum_diff_squared
        

#### * Testing the function on the example dataset `(total of 10 frames with window size 2)`

In [304]:
x = steadiness(data, 2)

In [311]:
import time
start_time = time.time()

x = steadiness(data, 2)

print("---This task takes %s seconds to run---" % (time.time() - start_time))


---This task takes 0.09705114364624023 seconds to run---


In [293]:
x

[4276382.0,
 8173670.5,
 2920876.5,
 2513283.5,
 2124824.5,
 1847891.0,
 1121319.0,
 1132827.0]

### 

#### * Testing the function on a bigger dataset `(total of 558 frames with window size of 10)`

In [200]:
y = steadiness(us_data, 10)

In [300]:
start_time = time.time()

y = steadiness(us_data, 10)

print("--- %s seconds ---" % (time.time() - start_time))

--- 29.29018497467041 seconds ---


In [202]:
len(y)

548

In [318]:
# Display the first 20 sum of squared differences
y[0:20]

[163400791.10000002,
 121323451.19999993,
 86982586.80000007,
 79051713.2,
 78218891.29999986,
 80517080.19999997,
 83597900.49999991,
 85328621.1999999,
 84831745.39999999,
 82147617.00000001,
 80932692.59999996,
 76394313.99999993,
 66632956.69999994,
 61185327.50000001,
 71139012.89999992,
 96673042.99999999,
 133270456.29999995,
 175718545.80000007,
 210046670.29999992,
 230418245.79999986]

##### 

#### * Testing the function on a bigger dataset `(total of 558 frames with window size of 30)`

In [ ]:
z = steadiness(us_data, 30)

In [325]:
start_time = time.time()

z = steadiness(us_data, 30)

print("--- %s seconds ---" % (time.time() - start_time))

--- 129.50022888183594 seconds ---


In [327]:
len(z)

528

In [326]:
# Display the first 20 sum of squared differences
z[0:20]

[938127781.9999993,
 937089004.2333343,
 946059751.266668,
 975945535.2333333,
 1007282633.566666,
 1037543443.9666642,
 1065046904.433333,
 1095671849.666667,
 1126486028.800001,
 1154060477.5999992,
 1178151562.0999992,
 1201057324.7333333,
 1225804493.833333,
 1259634291.9000022,
 1296281333.3333325,
 1330039829.3999991,
 1355095840.9333327,
 1361518388.9999986,
 1349696113.866665,
 1323833581.2333348]